In [1]:
import pandas as pd
import re

#### prepare fighter data 

In [2]:
data = pd.read_csv('../1_data_retrieval/ufc_fighter_info.csv')
data

,name,record,Height:,Weight:,Reach:,STANCE:,DOB:,SLpM:,Str. Acc.:,SApM:,Str. Def:,Unnamed: 11,TD Avg.:,TD Acc.:,TD Def.:,Sub. Avg.:,url
0,Lerone Murphy,Record: 14-0-1,"5' 9""",145 lbs.,"73""",Orthodox,"Jul 22, 1991",5.01,54%,2.60,60%,NaN,1.59,52%,46%,0.6,http://www.ufcstats.com/fighter-details/396fe8...
1,Edson Barboza,Record: 24-12-0,"5' 11""",145 lbs.,"75""",Orthodox,"Jan 21, 1986",4.11,44%,4.65,56%,NaN,0.45,50%,72%,0.1,http://www.ufcstats.com/fighter-details/64a50d...
2,Khaos Williams,Record: 15-3-0,"6' 0""",170 lbs.,"77""",Orthodox,"Mar 30, 1994",5.81,39%,5.42,42%,NaN,0.00,0%,80%,0.0,http://www.ufcstats.com/fighter-details/2558ae...
3,Carlston Harris,Record: 19-6-0,"6' 0""",170 lbs.,"76""",Orthodox,"Jul 09, 1987",3.18,49%,2.66,52%,NaN,1.97,29%,55%,0.8,http://www.ufcstats.com/fighter-details/a53d30...
4,Themba Gorimbo,Record: 13-4-0,"6' 1""",170 lbs.,"77""",Orthodox,"Jan 23, 1991",2.53,69%,1.08,46%,NaN,4.04,58%,78%,0.0,http://www.ufcstats.com/fighter-details/40f3cb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2493,Frank Hamaker,Record: 1-0-0,--,--,--,NaN,--,0.00,0%,0.00,0%,NaN,0.00,0%,0%,0.0,http://www.ufcstats.com/fighter-details/c3c23c...
2494,Thaddeus Luster,Record: 0-1-0,"6' 3""",210 lbs.,--,NaN,--,0.00,0%,0.00,0%,NaN,0.00,0%,0%,0.0,http://www.ufcstats.com/fighter-details/505934...
2495,David Levicki,Record: 1-3-0,"6' 5""",275 lbs.,--,NaN,--,0.00,0%,0.00,0%,NaN,0.00,0%,0%,0.0,http://www.ufcstats.com/fighter-details/49590e...
2496,Ray Wizard,Record: 0-1-0,--,--,--,NaN,--,0.00,0%,0.00,0%,NaN,0.00,0%,0%,0.0,http://www.ufcstats.com/fighter-details/ea0ad1...


In [3]:
# Convert to centimeters
def inches_to_cm(height_str):
    # Use regular expression to extract feet and inches
    match = re.match(r"(\d+)\'\s*(\d+)\"|(\d+)\"", height_str)
    if match:
        feet = int(match.group(1)) if match.group(1) else 0
        inches = int(match.group(2)) if match.group(2) else int(match.group(3))
        total_inches = feet * 12 + inches
        return total_inches * 2.54
    else:
        return None

In [4]:
data.drop(data.columns[[1, 6, 11, 16]], axis=1, inplace=True)
data.rename(columns=lambda x: x.rstrip(':'), inplace=True)
data.columns = data.columns.str.lower()
data.columns = ['name', 'height', 'weight', 'reach', 'stance', 'slpm', 'str_acc_%', 'sapm', 'str_def_%', 'td_avg', 'td_acc_%', 'td_def_%', 'sub_avg']
data['height'] = data['height'].apply(inches_to_cm)
data['weight'] = data['weight'].apply(lambda x: x.split()[0])
data['reach'] = data['reach'].apply(lambda x: x.split('"')[0])
data['str_acc_%'] = data['str_acc_%'].apply(lambda x: x.split('%')[0])
data['str_def_%'] = data['str_def_%'].apply(lambda x: x.split('%')[0])
data['td_acc_%'] = data['td_acc_%'].apply(lambda x: x.split('%')[0])
data['td_def_%'] = data['td_def_%'].apply(lambda x: x.split('%')[0])
data['reach'] = data['reach'].replace('--', None)
data['stance'].fillna(data['stance'].mode()[0], inplace=True)
data['stance'] = data['stance'].replace(['Southpaw', 'Switch', 'Open Stance', 'Sideways'], 'Unorthodox')
columns_to_convert = ['height', 'weight', 'reach', 'slpm', 'str_acc_%', 'sapm', 'str_def_%', 'td_avg', 'td_acc_%', 'td_def_%', 'sub_avg']
data[columns_to_convert] = data[columns_to_convert].apply(pd.to_numeric, errors='coerce')
data['height'].fillna(data['height'].mean(), inplace=True)
data['weight'].fillna(data['weight'].mean(), inplace=True)
# Define the desired column order
desired_columns = ['name', 'height', 'weight', 'reach', 'slpm', 'str_acc_%', 'sapm',
                   'str_def_%', 'td_avg', 'td_acc_%', 'td_def_%', 'sub_avg', 'stance']
# Rearrange the columns
data = data.reindex(columns=desired_columns)
data

,name,height,weight,reach,slpm,str_acc_%,sapm,str_def_%,td_avg,td_acc_%,td_def_%,sub_avg,stance
0,Lerone Murphy,175.260000,145.000000,73.0,5.01,54,2.60,60,1.59,52,46,0.6,Orthodox
1,Edson Barboza,180.340000,145.000000,75.0,4.11,44,4.65,56,0.45,50,72,0.1,Orthodox
2,Khaos Williams,182.880000,170.000000,77.0,5.81,39,5.42,42,0.00,0,80,0.0,Orthodox
3,Carlston Harris,182.880000,170.000000,76.0,3.18,49,2.66,52,1.97,29,55,0.8,Orthodox
4,Themba Gorimbo,185.420000,170.000000,77.0,2.53,69,1.08,46,4.04,58,78,0.0,Orthodox
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2493,Frank Hamaker,178.341429,169.190726,NaN,0.00,0,0.00,0,0.00,0,0,0.0,Orthodox
2494,Thaddeus Luster,190.500000,210.000000,NaN,0.00,0,0.00,0,0.00,0,0,0.0,Orthodox
2495,David Levicki,195.580000,275.000000,NaN,0.00,0,0.00,0,0.00,0,0,0.0,Orthodox
2496,Ray Wizard,178.341429,169.190726,NaN,0.00,0,0.00,0,0.00,0,0,0.0,Orthodox


In [5]:
data.isna().sum()

name           0
height         0
weight         0
reach        655
slpm           0
str_acc_%      0
sapm           0
str_def_%      0
td_avg         0
td_acc_%       0
td_def_%       0
sub_avg        0
stance         0
dtype: int64

In [6]:
from sklearn.linear_model import LinearRegression
reach_data = data[['height','reach']].copy()
pred_data = reach_data[reach_data['reach'].isna()]

# Split data into training and prediction sets
train_data = reach_data.dropna()
f2_data = data[['height', 'reach']].copy()

# Train the linear regression model
X_train = train_data[['height']]
y_train = train_data['reach']

model = LinearRegression()
model.fit(X_train, y_train)

# Predict missing values
X_pred = pred_data[['height']]
pred_values = model.predict(X_pred)

# Replace missing values with predicted values
data.loc[data['reach'].isna(), 'reach'] = pred_values

In [7]:
data.isna().sum().sum()

0

In [8]:
fighter_data = data.copy()

#### add clustering 

In [9]:
import pickle

# Unpickle the scaler
with open('../scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Unpickle the dummy variables
with open('../dummy_variables.pkl', 'rb') as f:
    dummy_variables = pickle.load(f)

# Unpickle the cluster labels
with open('../cluster_labels.pkl', 'rb') as f:
    cluster_labels = pickle.load(f)

# Load the KMeans model
with open('../kmeans_model.pkl', 'rb') as f:
    kmeans_model = pickle.load(f)

In [10]:
cluster_data = data[['reach', 'stance', 'slpm', 'str_acc_%', 'sapm', 'str_def_%', 'td_avg', 'td_acc_%', 'td_def_%', 'sub_avg']]
cluster_data.head(2)

,reach,stance,slpm,str_acc_%,sapm,str_def_%,td_avg,td_acc_%,td_def_%,sub_avg
0,73.0,Orthodox,5.01,54,2.60,60,1.59,52,46,0.6
1,75.0,Orthodox,4.11,44,4.65,56,0.45,50,72,0.1


In [16]:
from sklearn.cluster import KMeans

# Load your dataset
data = cluster_data

data = pd.get_dummies(data, columns=['stance'], drop_first=True)

# Separate numerical features for scaling
numerical_features = data.select_dtypes(include=['float64', 'int64']).columns
X_numerical = data[numerical_features]

# Scale numerical features using the loaded scaler
scaled_numerical = scaler.transform(X_numerical)

# Combine scaled numerical features with encoded categorical features
preprocessed_data = pd.concat([pd.DataFrame(scaled_numerical, columns=numerical_features), data.drop(columns=numerical_features)], axis=1)

# Use the KMeans model to predict clusters for the preprocessed data
predicted_clusters = kmeans_model.predict(preprocessed_data)

# Add the predicted clusters to the data DataFrame
fighter_data['cluster'] = predicted_clusters

In [17]:
fighter_data

,name,height,weight,reach,slpm,str_acc_%,sapm,str_def_%,td_avg,td_acc_%,td_def_%,sub_avg,stance,cluster
0,Lerone Murphy,175.260000,145.000000,73.000000,5.01,54,2.60,60,1.59,52,46,0.6,Orthodox,5
1,Edson Barboza,180.340000,145.000000,75.000000,4.11,44,4.65,56,0.45,50,72,0.1,Orthodox,5
2,Khaos Williams,182.880000,170.000000,77.000000,5.81,39,5.42,42,0.00,0,80,0.0,Orthodox,3
3,Carlston Harris,182.880000,170.000000,76.000000,3.18,49,2.66,52,1.97,29,55,0.8,Orthodox,0
4,Themba Gorimbo,185.420000,170.000000,77.000000,2.53,69,1.08,46,4.04,58,78,0.0,Orthodox,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2493,Frank Hamaker,178.341429,169.190726,71.972067,0.00,0,0.00,0,0.00,0,0,0.0,Orthodox,2
2494,Thaddeus Luster,190.500000,210.000000,77.028536,0.00,0,0.00,0,0.00,0,0,0.0,Orthodox,2
2495,David Levicki,195.580000,275.000000,79.141191,0.00,0,0.00,0,0.00,0,0,0.0,Orthodox,2
2496,Ray Wizard,178.341429,169.190726,71.972067,0.00,0,0.00,0,0.00,0,0,0.0,Orthodox,2


In [19]:
fighter_data['cluster'].value_counts()

cluster
2    981
0    771
5    565
3    171
1     10
Name: count, dtype: int64

In [20]:
fighter_data

,name,height,weight,reach,slpm,str_acc_%,sapm,str_def_%,td_avg,td_acc_%,td_def_%,sub_avg,stance,cluster
0,Lerone Murphy,175.260000,145.000000,73.000000,5.01,54,2.60,60,1.59,52,46,0.6,Orthodox,5
1,Edson Barboza,180.340000,145.000000,75.000000,4.11,44,4.65,56,0.45,50,72,0.1,Orthodox,5
2,Khaos Williams,182.880000,170.000000,77.000000,5.81,39,5.42,42,0.00,0,80,0.0,Orthodox,3
3,Carlston Harris,182.880000,170.000000,76.000000,3.18,49,2.66,52,1.97,29,55,0.8,Orthodox,0
4,Themba Gorimbo,185.420000,170.000000,77.000000,2.53,69,1.08,46,4.04,58,78,0.0,Orthodox,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2493,Frank Hamaker,178.341429,169.190726,71.972067,0.00,0,0.00,0,0.00,0,0,0.0,Orthodox,2
2494,Thaddeus Luster,190.500000,210.000000,77.028536,0.00,0,0.00,0,0.00,0,0,0.0,Orthodox,2
2495,David Levicki,195.580000,275.000000,79.141191,0.00,0,0.00,0,0.00,0,0,0.0,Orthodox,2
2496,Ray Wizard,178.341429,169.190726,71.972067,0.00,0,0.00,0,0.00,0,0,0.0,Orthodox,2


In [21]:
fighter_data.to_csv('../clustered_fighter.csv', index=False)